In [1]:
import tf
import rospy
import json
import rospkg
import transformations

In [2]:
def frame_publisher(Cskeleton, sensor, Kskeleton=None):
    tfb = tf.TransformBroadcaster()
    rospy.sleep(1)
    rate = rospy.Rate(10)
    while not rospy.is_shutdown():
        for key in Cskeleton.keys():
            if Kskeleton is not None:
                tfb.sendTransform(Kskeleton[key][0], Kskeleton[key][1], rospy.Time.now(),
                                  '/human/'+key, '/optitrack_frame')    
            tfb.sendTransform(Cskeleton[key][0], Cskeleton[key][1], rospy.Time.now(),
                              '/human/'+key+'_calibr', '/optitrack_frame')
            rate.sleep()

In [3]:
rospy.init_node('test_calibration')

In [4]:
rospack = rospkg.RosPack()

In [5]:
sensor = 'opt'

In [61]:
sensor = 'kinect'

In [6]:
skeleton_file = rospack.get_path("human_moveit_config")+'/tmp/skeleton.json'
# read the T pose from human
with open(skeleton_file) as data_file:
    data_skeletons = json.load(data_file)
skeleton = data_skeletons[sensor]

In [7]:
calibration_file = rospack.get_path("human_moveit_config")+'/tmp/sensor_calibration.json'
# read the T pose from human
with open(calibration_file) as data_file:
    data_calibration = json.load(data_file)
calibration = data_calibration[sensor]

In [8]:
KtB = skeleton['base']

In [9]:
# convert back all skeletons frames into kinect
Kskeleton = {}
for key, value in skeleton.iteritems():
    if key != 'base':
        Kskeleton[key] = transformations.multiply_transform(KtB, value)
    else:
        Kskeleton[key] = value

In [10]:
# convert all skeleton data to their calibrated values
Cskeleton = {}
base_transform = transformations.inverse_transform(calibration['base'])
for key, value in calibration.iteritems():
    Cskeleton[key] = transformations.multiply_transform(Kskeleton[key], value)

In [24]:
frame_publisher(Cskeleton, sensor)

KeyboardInterrupt: 

In [11]:
frame_publisher(Cskeleton, sensor, Kskeleton)

KeyboardInterrupt: 